In [1]:
from typing import runtime_checkable, Protocol
import contextvars

In [2]:
@runtime_checkable
class PackageManager(Protocol):
    def install(self, package: str) -> None:
        pass

class Pip(PackageManager):
    def install(self, package: str) -> None:
        print(f"pip install {package}")

class Conda(PackageManager):
    def install(self, package: str) -> None:
        print(f"conda install {package}")

In [3]:
@runtime_checkable
class DependancyWrapperProtocol(Protocol):
    
    @classmethod
    def get_dependencies(cls) -> list[str]:
        pass

    @classmethod
    def install_additional() -> None:
        pass

In [4]:
current_context_manager = contextvars.ContextVar("current_context_manager", default=None)

@runtime_checkable
class Runtime(Protocol):
  
   def prepare(self, components: set[DependancyWrapperProtocol]) -> None:
       raise NotImplementedError

   def register(self, component: DependancyWrapperProtocol) -> None:
       raise NotImplementedError


class LocalPythonRuntime:
    def __init__(self, package_manager: PackageManager = None):
        self.components: set[DependancyWrapperProtocol] = set()
        self.package_manager = package_manager or Pip()

    def register(self, component: DependancyWrapperProtocol) -> None:
        self.components.add(type(component))

    def prepare(self, components: set[DependancyWrapperProtocol]) -> None:
        for component in components:
            for dependency in component.get_dependencies():
                self.package_manager.install(dependency)
            component.install_additional()

    def __enter__(self):
        self.token = current_context_manager.set(self)
        return self
    
    def __exit__(self, exc_type, exc_value, traceback):
        self.prepare(self.components)
        current_context_manager.reset(self.token)

class RuntimeAware:
   def __init__(self):
       # Check if we're inside a context manager by getting the current value from contextvar
       context_manager = current_context_manager.get(None)
       if context_manager is not None:
           context_manager.register(self)

In [5]:
class MyComponent(RuntimeAware):
    @classmethod
    def get_dependencies(cls) -> list[str]:
        return ['requests']
    
    @classmethod
    def install_additional(cls) -> None:
        print("Installing additional dependencies")

In [6]:
with LocalPythonRuntime() as runtime:
    component = MyComponent()

pip install requests
Installing additional dependencies


In [7]:
with LocalPythonRuntime():
    component = MyComponent()
    with LocalPythonRuntime(package_manager=Conda()):
        component_conda = MyComponent()

conda install requests
Installing additional dependencies
pip install requests
Installing additional dependencies
